In [1]:
import schedule
import requests
import time
import json
import pandas as pd
import copy
# Accessing dictionary inside the list
def createdf(list1):
    subdf = pd.DataFrame(columns=['StateandUT','Dates','Confirmed','Recovered','Deceased'])
    df = pd.DataFrame(columns=['StateandUT','Dates','Confirmed','Recovered','Deceased'])
    count = 0
    l2 = copy.deepcopy(list1)
    # Take each element of the json
    for tempdict in l2:
        # Preparing columns date, state, Status-Confirm, dead, cured
        dt = tempdict['date']
        st = tempdict['status']
        
        # Removing date and status
        del tempdict['date']
        del tempdict['status']
        
        # Using remaining values to fill the field as per the date and status
        if(st=='Confirmed'):
            subdf['Confirmed'] = tempdict.values()
            subdf['Dates'] = dt
            subdf['StateandUT'] = tempdict.keys()
            #statelist = tempdict.keys() 
        elif(st=='Recovered'):
            subdf['Recovered'] = tempdict.values()
            #statelist = tempdict.keys()
        elif(st=='Deceased'):
            subdf['Deceased'] = tempdict.values()
        count+=1
        if(count==3):
            df = df.append(subdf)
            count = 0
    df.reset_index(drop=True,inplace=True)
    # Converting the lower case initials of state to upper case
    df['StateandUT'] = df['StateandUT'].str.upper()
    # Replacing initials with full name
    df.replace({'StateandUT':
               {"AN":"Andaman and Nicobar","AP":"Andhra Pradesh","AR":"Arunachal Pradesh","AS":"Assam",
                "BR":"Bihar","CH":"Chandigarh","CT":"Chhattisgarh","DN":"Dadra and Nagar Haveli",
                "DD":"Daman and Diu","DL":"Delhi","GA":"Goa","GJ":"Gujarat","HR":"Haryana","HP":"Himachal Pradesh",
                "JK":"Jammu and Kashmir","JH":"Jharkhand","KA":"Karnataka","KL":"Kerala","LA":"Ladakh",
                "LD":"Lakshadweep","MP":"Madhya Pradesh","MH":"Maharashtra","MN":"Manipur","ML":"Meghalaya",
                "MZ":"Mizoram","NL":"Nagaland","OR":"Odisha","PY":"Puducherry","PB":"Punjab","RJ":"Rajasthan",
                "SK":"Sikkim","TN":"Tamil Nadu","TG":"Telangana","TR":"Tripura","UP":"Uttar Pradesh",
                "UT":"Uttarakhand","WB":"West Bengal"}},inplace=True)
    
    #ind = list(df.loc[df['StateandUT']=='TT',:].index)
    #ind.extend(list(df.loc[df['StateandUT']=='UN',:].index))
    #df['StateandUT'].drop(ind,inplace=True)
    return df

# Function to get the data from url
def pull_data(site_url,file_name):
    global r
    r = requests.get(url=site_url)
    file = open(file_name, "a")
    
    # Convert obtained json into a list
    obj = r.json()
    
    # Create a dataframe of the obtained list from json
    output = createdf(obj['states_daily'])
    output.to_csv(file_name)

In [ ]:
url = 'https://api.covid19india.org/states_daily.json'
file_name = 'F:/Aditya/Youtube content/COVID19India.csv'

# Scheduling to get the updated data
schedule.every(1).minutes.do(pull_data, url, file_name)
while True:
    schedule.run_pending()
    time.sleep(120)